# COGS 108 - Assignment 3: Data Privacy

## Important Reminders

- Rename this file to 'A3_$####.ipynb', replacing with your unique ID (first letter of your last name, followed by the last 4 digits of your student ID number), before you submit it. Submit it to TritonED.
- Do not change / update / delete any existing cells with 'assert' in them. These are the tests used to check your assignment. 
    - Changing these will be flagged for attempted cheating. 
- This assignment has hidden tests: tests that are not visible here, but that will be run on your submitted file. 
    - This means passing all the tests you can see in the notebook here does not guarantee you have the right answer!

## Overview

We have discussed in lecture the importance and the mechanics of protecting individuals privacy when they are included in datasets. 

One method to do so is the Safe Harbor Method. The Safe Harbour method specifies how to protect individual's identities by telling us which tells us which information to remove from a dataset in order to avoid accidently disclosing personal information. 

In this assignment, we will explore web scraping, which can often include personally identifiable information, how identity can be decoded from badly anonymized datasets, and also explore using Safe Harbour to anonymize datasets properly. 

The topics covered in this assignment are mainly covered in the 'DataGathering' and 'DataPrivacy&Anonymization' Tutorial notebooks.

### Installing new packages

In the first part of the assignment we will understand how we can scrape the web for data. You have to use the Beautiful Soup library in Python for scraping the data. 

The library is not installed in Anaconda version, therefore to install a new library for Anaconda, we can use the conda package manager, which the cell below does for you. 

In [38]:
# Run this cell to install beautifulsoup4
#  You only need to do the installation once
#    Once you have run it you can comment these two lines so that the cell doesn't execute everytime.
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

Solving environment: ...working... done

# All requested packages already installed.



### Imports

In [39]:
# Imports - these provided for you. Do not import any other packages
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## Part 1: Web Scraping 

### Scraping Rules

1) If you are using another organizations website for scraping, make sure to check the website's terms & conditions. 

2) Do not request data from the website too aggressively (quickly) with your program (also known as spamming), as this may break the website. Make sure your program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.

3) The layout of a website may change from time to time. Because of this, if you're scraping website, make sure to revisit the site and rewrite your code as needed.

In [40]:
# This cell will help you understand the permission issues related to accessing a page
# Uncomment the two lines, run them, see what error you get, comment them again

#page_source = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')
#page_soup = BeautifulSoup(page_source.content, 'html.parser')

In [41]:
# This cell will help you understand the permission issues related to accessing a page
# Uncomment the two lines, run them, see what error you get, comment them again

page_source = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')
page_soup = BeautifulSoup(page_source.content, 'html.parser')

#### What is the error that you got, and why did you get it?

YOUR ANSWER HERE

In [42]:
# 1a) Web Scrape
# We will first retrieve the contents on a page and examine them a bit.

# Make a variable called 'wiki', that stores the following URL (as a string):
#  'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
# To open the URL you can use 'requests.get' as shown in the cell above. Call this variable 'page'
# After that use BeautifulSoup Library to open the URL and assign it to an object called 'soup'

# YOUR CODE HERE
wiki = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
page = requests.get(wiki)
soup = BeautifulSoup(page.content, 'html.parser')
#raise NotImplementedError()

In [43]:
assert wiki
assert page
assert soup


In [44]:
# 1b) Checking Scrape Contents

# Extract the title from the page and save it in a variable called 'title_page'. 
#  Make sure you extract it as a string.
# To do so, you have to use the soup object created in the above cell. 
#  Hint: from your soup variable, you can access this with '.title.string'
# Make sure you print out and check the contents of 'title_page'. 
#  Note that it should not have any tags (such as '<title>' included in it).

# YOUR CODE HERE
title_page = soup.title.string
#title_page
#raise NotImplementedError()

In [45]:
assert title_page
assert isinstance(title_page, str)


In [46]:
# 1c) Extracting Tables

# In order to extract the data we want, we'll start with extracting a data table of interest. 
#  Note that you can see this table by going to look at the link we scraped.
# Use the soup object and call a method called find, which will and extract the first table in scraped webpage. 
#  Note: you need to search for the name 'table', and set the 'class_' argument as 'wikitable sortable'.

# YOUR CODE HERE
right_table = soup.find('table')
#right_table
#raise NotImplementedError()

In [47]:
assert right_table
assert isinstance(right_table, bs4.element.Tag)
assert right_table.name == 'table'

In [48]:
# Extract the data from the table into lists.
#  Note: This code provided for you. Do read through it and try to see how it works.

lst_a, lst_b, lst_c = [], [], []

for row in right_table.findAll('tr'):
    
    cells = row.findAll('td')
    
    # Skips rows that aren't 10 columns long (like the heading)
    if len(cells) != 10:
        continue

    # This catches when the name cells stops having a link
    #  and ends, skipping the last (summary rows)
    try:
        lst_a.append(cells[2].find('a').text)
        lst_b.append(cells[4].find(text=True))
        lst_c.append(cells[5].find(text=True))
    except:
        break

In [49]:
# 1d) Collecting into a dataframe

# Create a dataframe 'my_df' and add the data from the lists above to it. 
#  'lst_a' is the state or territory name. Set the column name as 'State', and make this the index
#  'lst_b' is the population estimate. Add it to the dataframe, and set the column name as 'Population Estimate'
#  'lst_c' is the census population. Add it to the dataframe, and set the column name as 'Census Population'

# YOUR CODE HERE
#my_df = pd.DataFrame({'State': lst_a,
#                      'Population Estimate': lst_b,
#                      'Census Population': lst_c})
my_df = pd.DataFrame()
my_df['State'] = lst_a
my_df['Population Estimate'] = lst_b
my_df['Census Population'] = lst_c

my_df.set_index('State', inplace=True)
#my_df

#raise NotImplementedError()

In [50]:
assert isinstance (my_df, pd.DataFrame)
assert my_df.index.name == 'State'
assert list(my_df.columns) == ['Population Estimate', 'Census Population']


In [51]:
# 1e) Using the data
# What is the Population Estimate of California? Save this answer to a variable called 'calif_pop'
# Notes:
#  Extract this value programmatically from your dataframe (as in, don't set it explicitly, as 'cf = 123')
#    You can use '.loc' to extract a particular value from a dataframe.
#  The data in your dataframe will be strings - that's fine, leave them as strings (don't typecast).

# YOUR CODE HERE
calif_pop = my_df.loc['California', 'Population Estimate']
#calif_pop
#raise NotImplementedError()

In [52]:
assert calif_pop


## Part 2: Identifying Data

Data Files:
- anon_user_dat.json
- employee_info.json

You will first be working with a file called 'anon_user_dat.json'. This file that contains information about some (fake) Tinder users. When creating an account, each Tinder user was asked to provide their first name, last name, work email (to verify the disclosed workplace), age, gender, phone # and zip code. Before releasing this data, a data scientist cleaned the data to protect the privacy of Tinder's users by removing the obvious personal identifiers: phone #, zip code, and IP address. However, the data scientist chose to keep each users' email addresses because when they visually skimmed a couple of the email addresses none of them seemed to have any of the user's actual names in them. This is where the data scientist made a huge mistake!

We will take advantage of having the work email addresses by finding the employee information of different companies and matching that employee information with the information we have, in order to identify the names of the secret Tinder users!

In [53]:
# 2a) Load in the 'cleaned' data 

# Load the json file into a pandas dataframe. Call it 'df_personal'.

# YOUR CODE HERE
df_personal = pd.read_json('anon_user_dat.json')
#raise NotImplementedError()

In [54]:
assert isinstance(df_personal, pd.DataFrame)


In [55]:
# 2b) Check the first 10 emails 

# Save the first 10 emails to a Series, and call it 'sample_emails'. 
# You should then and print out this Series. 
# The purpose of this is to get a sense of how these work emails are structured
#   and how we could possibly extract where each anonymous user seems to work.

# YOUR CODE HERE
s = []
for i in range(10):
    s.append(df_personal.email[i])
sample_emails = pd.Series(s)
#sample_emails
#raise NotImplementedError()

In [56]:
assert isinstance(sample_emails, pd.Series)


In [57]:
# 2c) Extract the Company Name From the Email 

# Create a function with the following specifications:
#   Function Name: extract_company
#   Purpose: to extract the company of the email 
#          (i.e., everything after the @ sign but before the .)
#   Parameter(s): email (string)
#   Returns: The extracted part of the email (string)
#   Hint: This should take 1 line of code. Look into the find('') method. 
#
# You can start with this outline:
#   def extract_company(email):
#      return 
#
# Example Usage: 
#   extract_company("larhe@uber.com") should return "uber"
#   extract_company(“ds@cogs.edu”) should return “cogs”


# YOUR CODE HERE
#s="larhe@uber.com"
#start = s.find('@')+1
#end = s.find('.')
#temp = s[start:end]
#temp

def extract_company(x):
    start = x.find('@')+1
    end = x.find('.')
    temp = x[start:end]
    return temp
    

#raise NotImplementedError()

In [58]:
assert extract_company("gshoreson0@seattletimes.com") == "seattletimes"


With a little bit of basic sleuthing (aka googling) and web-scraping (aka selectively reading in html code) it turns out that you've been able to collect information about all the present employees/interns of the companies you are interested in. Specifically, on each company website, you have found the name, gender, and age of its employees. You have saved that info in employee_info.json and plan to see if, using this new information, you can match the Tinder accounts to actual names.

In [59]:
# 2d) Load in employee data 

# Load the json file into a pandas dataframe. Call it 'df_employee'.

# YOUR CODE HERE
#df=df_employee.loc[(df_employee['company'] == company) & (condition2) & (condition3)]
#first_name = list(df['first_name'])
df_employee = pd.read_json('employee_info.json')

#raise NotImplementedError()

In [60]:
assert isinstance(df_employee, pd.DataFrame)


In [61]:
# 2e) Match the employee name with company, age, gender 

# Create a function with the following specifications:
#   Function name: employee_matcher
#   Purpose: to match the employee name with the provided company, age, and gender
#   Parameter(s): company (string), age (int), gender (string)
#   Returns: The employee first_name and last_name like this: return first_name, last_name 
#   Note: If there are multiple employees that fit the same description, first_name and 
#         last_name should return a list of all possible first names and last name
#         i.e., ['Desmund', 'Kelby'], ['Shepley', 'Tichner']
#
# Hint:
# There are many different ways to code this.
# 1) An unelegant solution is to loop through df_employee 
#    and for each data item see if the company, age, and gender match
#    i.e., for i in range(0, len(df_employee)):
#              if (company == df_employee.ix[i,'company']):
#
# However! The solution above is very inefficient and long, 
# so you should try to look into this:
# 2) Google the df.loc method: It extracts pieces of the dataframe
#    if it fulfills a certain condition.
#    i.e., df_employee.loc[df_employee['company'] == company]
#    If you need to convert your pandas data series into a list,
#    you can do list(result) where result is a pandas "series"
# 
# You can start with this outline:
#   def employee_matcher(company, age, gender):
#      return first_name, last_name

# YOUR CODE HERE
#df=df_employee.loc[(df_employee['company'] == company) & (condition2) & (condition3)]
#first_name = list(df['first_name'])
#df_employee
def employee_matcher(company, age, gender):
    df=df_employee.loc[(df_employee['company'] == company) & (df_employee['age'] == age) & (df_employee['gender'] == gender)]
    first_name = list(df['first_name'])
    last_name = list(df['last_name'])
    return first_name, last_name
        
#df_personal

#raise NotImplementedError()

In [62]:
assert employee_matcher("google", 41, "Male") == (['Maxwell'], ['Jorio'])
assert employee_matcher("salon", 47, "Female") == (['Elenore'], ['Gravett'])


In [63]:
# 2f) Extract all the private data 

# - Create 2 empty lists called 'first_names' and 'last_names'
# - Loop through all the people we are trying to identify in df_personal
# - Call the extract_company function (i.e., extract_company(df_personal.ix[i, 'email']) )
# - Call the employee_matcher function 
# - Append the results of employee_matcher to the appropriate lists (first_names and last_names)

# YOUR CODE HERE
first_names = []
last_names = []

#for i in df_personal.index:
    #age = df_personal.ix([i, 'age'])
    #company = extract_company(df_personal.ix[i, 'email'])
    #gender = df_personal.ix([i, 'gender'])
    #first, last = employee_matcher(company, age, gender)
    #first_names.append(first)

for i in df_personal.index:
    company = extract_company(df_personal.ix[i, 'email'])
    first, last = employee_matcher(company, df_personal.ix[i, 'age'], df_personal.ix[i, 'gender'])
    first_names.append(first)
    last_names.append(last)
    
#first_names

#first, last = employee_matcher(company, age, gender)
#list_a.append(first)

#raise NotImplementedError()

C:\Users\vince\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [64]:
assert first_names[45:50]== [['Justino'], ['Tadio'], ['Kennith'], ['Cedric'], ['Amargo']]
assert last_names[45:50] == [['Corro'], ['Blackford'], ['Milton'], ['Yggo'], ['Grigor']]


In [65]:
# 2g) Add the names to the original 'secure' dataset! 

# We have done this last step for you below, all you need to do is run this cell.
# For your own personal enjoyment, you should also print out
#   the new df_personal with the identified people. 

df_personal['first_name'] = first_names
df_personal['last_name'] = last_names

We have now just discovered the 'anonymous' identities of all the registered Tinder users...awkward.

## Part 3: Anonymize Data

You are hopefully now convinced that with some seemingly harmless data a hacker can pretty easily discover the identities of certain users. Thus, we will now clean the original Tinder data ourselves according to the Safe Harbor Method in order to make sure that it has been *properly* cleaned...

In [66]:
# 3a) Load in personal data 

# Load the user_dat.json file into a pandas dataframe. Call it 'df_users'.
# Note: You might find that using the same method as A2 (or above) leads to an error.
# The file has a slightly different organization. 
#   Try googling the error and finding the fix for it.
# Hint: you can still use 'pd.read_json', you just need to add another argument.

# YOUR CODE HERE
df_users = pd.read_json('user_dat.json', lines=True)

#raise NotImplementedError()

In [67]:
assert isinstance(df_users, pd.DataFrame)


In [68]:
# 3b) Drop personal attributes 

# Remove any personal information, following the Safe Harbour method.
# Based on the Safe Harbour method, remove any columns from df_users that contain personal information.
#   Note that details on the Safe Harbour method are covered in the Tutorials.

# YOUR CODE HERE
cols = ['age','email','gender']
df_users = df_users[cols]
#df_users
#raise NotImplementedError()

In [69]:
assert len(df_users.columns) == 3


In [73]:
# 3c) Drop ages that are above 90 

# Safe Harbour rule C:
#   Drop all the rows which have age greater than 90 from df_users

# YOUR CODE HERE
df_users = df_users[df_users['age'] <= 90]
#df_users.shape
#raise NotImplementedError()

(993, 3)

In [72]:
assert df_users.shape == (993, 3)


In [76]:
# 3d) Load in zip code data 

# Load the zip_pop.csv file into a (different) pandas dataframe. Call it 'df_zip'.
# Note that the zip data should be read in as strings, not ints, as would be the default. 
# In read_csv, use the parameter 'dtype' to specify to read 'zip' as str, and 'population' as int.

# YOUR CODE HERE
df_zip = pd.read_csv('zip_pop.csv')
#df_zip
#raise NotImplementedError()

In [77]:
assert isinstance(df_zip, pd.DataFrame)


In [104]:
# 3e) Sort zipcodes into "Geographic Subdivision" 

# The Safe Harbour Method applies to "Geographic Subdivisions"
#   as opposed to each zipcode itself. 
# Geographic Subdivision:
#   All areas which share the first 3 digits of a zip code
#
# Count the total population for each geographic subdivision
# Warning: you have to be savy with a dictionary here
# To understand how a dictionary works, check the section materials,
#   use google and go to discussion sections!
#
# Instructions: 
# - Create an empty dictionary: zip_dict = {}
# - Loop through all the zip_codes in df_zip
# - Create a dictionary key for the first 3 digits of a zip_code in zip_dict
# - Continually add population counts to the key that contains the 
#     same first 3 digits of the zip code
#
# To extract the population you will find this code useful:
#   population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
# To extract the first 3 digits of a zip_code you will find this code useful:
#   int(str(zip_code)[:3])
#
# Note: this code may take some time (many seconds, up to a minute or two) to run

# YOUR CODE HERE
#example
#10001- 20000
#10002- 5000
#10101- 2000
zip_dict = {}
zipcodes = []
for i in df_zip.zip:
    zipcodes.append(int(str(i)[:3]))
int zipper = zipcodes[0]
for x,y in zip(zipcodes,df_zip.population):
    if x == zipper:
        zip_dict[x] = 
    

#temp_dict = {x:y for x,y in zip(df_zip.zip,df_zip.population)}


#zip_dict = {}
#for i, rows in df

#zip_code = rows['zip_code']
#pop=rows['population']
#extract 1st 3 digits of your zip code -> key

#if key in zip_dict:
#    zip_dict[key] = zip_dict[key] + pop
#else:
#    zip_dict[key] = pop
    
#zip_dict[key].setdefault(0)
#zip_dict[key] = zip_dict[key] + pop






#raise NotImplementedError()

100 16769
100 29049
100 10372
100 5079
100 14649
100 1263
100 741
101 3609
101 1370
101 661
101 23188
102 29668
102 2451
102 946
102 17660
102 15720
102 789
103 11669
103 1308
103 570
103 6227
103 2021
103 5250
103 5109
103 838
103 2545
103 1336
104 39880
105 2530
105 1685
105 1851
105 21103
105 8534
106 15284
106 11150
106 430
106 64
106 1902
106 8469
107 629
107 1596
107 1478
107 5792
107 319
107 17527
107 9502
107 692
108 2334
108 1698
108 10322
108 2888
108 176
108 41117
108 687
108 670
108 28391
109 1394
109 325
109 351
109 14319
109 2217
109 111
109 1101
110 2479
110 22865
110 12350
110 16021
110 11611
110 26688
110 30250
111 14071
111 14152
112 2631
112 7019
115 8698
119 0
120 46504
122 8603
122 829
122 2314
122 182
122 3117
122 6945
122 95
123 8430
123 2766
123 1509
123 2496
123 6047
124 4534
124 476
124 334
124 151
124 985
124 16087
125 731
125 972
125 915
125 692
125 2121
125 374
125 635
126 240
126 1238
126 137
126 1463
126 7989
127 778
130 17700
133 1506
133 13263
133 2596


454 1152
454 1042
455 806
455 1752
455 710
455 1643
455 1249
455 308
456 2216
456 1534
456 489
456 391
457 2
457 323
457 5075
457 501
457 1510
457 85
457 604
457 4450
457 3072
460 12974
460 1272
460 1182
460 5222
461 508
461 589
461 183
461 2579
461 824
461 811
461 3121
462 1412
462 1054
462 184
462 47
462 507
462 1636
462 703
462 107
463 1398
463 1331
463 1923
463 61
463 136
464 2261
464 123
464 1018
464 13
464 73
464 94
464 580
464 1370
465 235
465 1690
465 490
465 3424
465 1119
465 363
465 1353
466 1309
466 488
466 1238
466 1172
466 1638
466 1680
466 204
467 576
467 112
467 484
467 253
467 1088
467 274
467 1767
468 1131
468 1043
468 104
468 1466
468 332
468 114
469 441
469 516
469 2376
473 9992
473 1670
473 221
473 800
473 617
473 9876
473 1112
474 1287
474 3
474 3496
474 4676
474 1082
474 470
474 1337
475 2603
475 3344
475 2870
475 1498
476 795
476 529
476 602
476 726
476 66
476 1262
476 386
476 400
476 9692
477 762
477 723
477 730
477 925
477 422
477 398
478 808
478 560
478 426
47

787 934
788 192
788 16
788 14492
788 10078
790 22806
792 26747
792 7621
792 12246
792 7534
792 120
792 3403
792 19144
793 8559
793 3296
793 9868
793 3251
793 1501
793 313
793 11157
793 228
794 17278
794 9539
794 3144
795 19564
796 43747
796 4
797 117
797 11768
797 754
797 669
797 589
798 2307
798 8865
800 844
800 22274
800 30629
800 12350
800 22448
800 518
800 5250
800 6975
800 12854
801 11539
801 33
801 38992
801 504
801 20763
801 33540
801 925
802 2201
802 44833
802 8783
802 451
802 2274
802 4888
802 20078
802 4784
803 13410
803 11584
803 16261
803 18151
803 7566
803 5917
803 24308
803 124
803 137
804 1002
804 203
804 29131
804 2909
804 31587
804 12495
804 5350
805 24285
805 10758
805 19172
805 45538
805 41864
805 27179
805 4736
805 21090
805 5612
806 24713
806 2672
806 15756
806 3102
806 311
806 7398
806 8437
806 1635
806 6727
806 13076
807 13059
807 9551
807 225
807 203
807 58
807 28894
807 18307
807 8385
807 10987
808 37433
808 50589
808 9455
808 7040
808 18737
808 7699
808 24104


121 1679
121 3168
121 295
121 1702
121 5137
121 20555
121 499
121 4569
121 2474
121 914
121 708
121 814
121 2818
121 1977
121 848
121 4061
121 6309
121 7896
121 944
121 160
121 365
121 175
121 1363
121 2438
121 1952
121 306
121 4945
121 405
121 1753
121 345
121 797
121 198
121 518
121 53606
121 14733
121 2620
121 7148
121 2037
121 6277
121 697
121 10980
121 17568
121 674
121 1722
121 2016
121 207
121 154
121 3031
121 2233
121 7903
122 9628
122 29952
122 7349
122 26977
122 16395
122 1948
122 20702
122 10121
122 10158
122 11303
122 6489
123 27582
123 29857
123 21482
123 5716
123 25636
123 7692
123 15335
123 29343
124 35040
124 3385
124 780
124 823
124 173
124 804
124 397
124 497
124 673
124 2948
124 10510
124 277
124 581
124 537
124 722
124 363
124 470
124 339
124 1097
124 250
124 576
124 6602
124 281
124 1194
124 1381
124 492
124 483
124 751
124 326
124 417
124 59
124 305
124 1980
124 114
124 953
124 3825
124 497
124 5061
124 388
124 3367
124 183
124 1674
124 213
124 366
124 297
124 198

144 9105
144 17813
144 6220
144 8016
144 2672
144 8598
144 234
144 2288
144 1732
144 1595
144 275
144 833
144 1820
144 8275
144 4267
144 308
144 6025
144 7192
145 10691
145 1701
145 5393
145 1325
145 1351
145 4895
145 497
145 4721
145 13976
145 6389
145 2365
145 2883
145 11581
145 2761
145 9565
145 2865
145 19804
145 12820
145 67
145 5575
145 4431
145 2000
145 31426
145 687
145 207
145 242
145 4129
145 97
145 3152
145 2101
145 173
145 4695
145 4249
145 91
145 1537
145 5651
145 974
145 18057
145 2164
145 3086
145 14487
145 6048
145 6211
145 1176
145 5081
145 50587
145 120
145 10256
145 749
145 7854
145 5446
145 1748
145 110
146 1743
146 12610
146 28255
146 16223
146 12268
146 42571
146 14233
146 17396
146 34515
146 14730
146 1224
146 16099
146 28534
146 22789
146 22920
146 14749
146 25141
146 33802
146 12108
146 27173
146 36296
146 10387
146 30041
146 2320
147 40730
147 6356
147 239
147 183
147 1535
147 3540
147 1766
147 3342
147 487
147 2747
147 3104
147 636
147 2163
147 3497
147 675
1

...

829 3654
829 227
829 1861
829 5028
829 123
829 301
829 216
829 351
830 15851
830 240
830 180
830 395
830 3595
830 400
831 3104
831 4307
831 54
831 465
831 666
831 3278
831 828
831 830
831 728
831 1139
831 172
831 112
831 359
831 14
831 494
831 711
831 137
831 334
831 3041
831 1601
832 37199
832 21608
832 309
832 18026
832 1175
832 3449
832 6374
832 137
832 1644
832 671
832 26
832 843
832 338
832 137
832 27021
832 139
832 2504
832 142
832 522
832 467
832 109
832 1213
832 99
832 2275
832 1146
832 109
832 572
832 1980
832 80
832 358
832 2257
832 1069
832 1637
832 1097
832 4154
832 165
832 3617
832 1213
832 536
832 1386
832 9258
832 531
832 313
832 9246
832 4097
832 111
832 186
832 93
832 147
832 131
832 1241
832 299
833 52010
833 196
833 562
833 143
833 25
833 3625
833 898
833 9626
833 17226
833 1006
833 550
833 42
833 1951
833 788
833 918
833 970
833 5347
833 6599
833 2361
833 10472
833 2060
833 1840
833 4951
833 33
833 19782
833 4682
833 6491
833 1298
833 73
833 1128
833 1442
833 3258
8

864 24289
864 15802
864 16243
864 23763
864 26471
864 224
864 12103
864 13863
864 7162
864 403
864 3933
864 111
864 1490
864 487
864 2104
864 76
864 913
864 6906
864 2224
864 33382
864 76
864 1289
864 311
865 1464
865 10714
865 5835
865 754
865 7682
865 1321
865 2340
865 752
865 5358
865 3694
865 2439
865 3011
865 306
865 2894
865 1585
865 1793
865 1199
865 2338
865 1088
865 1267
865 1650
865 1171
865 2090
870 3684
870 22517
870 10136
870 799
870 703
870 1080
870 2552
870 1139
870 84
870 261
870 5200
870 654
870 13105
870 3730
870 429
870 522
870 11347
870 3472
870 907
870 665
870 2214
870 1237
870 4063
870 309
870 145
870 237
870 44075
870 291
870 2207
870 7268
870 1868
870 677
870 616
870 496
870 881
870 3644
870 5052
870 387
870 1751
870 186
870 4894
870 8682
870 973
870 969
870 3182
870 998
870 954
870 9604
870 371
870 1869
870 323
870 161
870 4755
870 42
870 660
870 569
871 21204
871 12123
871 57680
871 27013
871 31183
871 38647
871 40432
871 38587
871 56490
871 43584
871 14477
87

921 39337
921 47490
921 51536
921 48940
921 32787
921 0
921 198
921 571
921 35125
921 3435
921 1449
921 559
921 79708
921 550
921 29429
922 61827
922 25605
922 4677
922 24294
922 31638
922 45
922 43605
922 24310
922 26234
922 2550
922 39073
922 8099
922 51151
922 41083
922 223
922 34722
922 8803
922 1539
922 48030
922 7334
922 8285
922 21583
922 9647
922 37262
922 12768
922 3588
922 2326
922 861
922 390
922 31753
922 26179
922 19383
922 234
922 126
922 574
922 17220
922 1823
922 19801
922 1506
922 7585
922 23911
922 3846
922 14
922 2432
922 1239
922 3158
922 25095
922 2632
923 32725
923 17
923 535
923 37630
923 39837
923 763
923 8845
923 31894
923 12025
923 10162
923 5094
923 30830
923 7788
923 1522
923 1257
923 56505
923 9391
923 632
923 445
923 65
923 472
923 95397
923 88419
923 37849
923 12
923 885
923 410
923 6379
923 20769
923 78715
923 54923
923 1692
923 8004
923 21559
923 6455
923 707
923 8103
923 5321
923 90
923 2637
923 1113
923 16763
923 6220
923 33423
923 40267
923 81516
923

955 3450
955 652
955 462
955 1071
955 1930
955 136
955 361
955 2579
955 271
955 1744
955 508
955 201
955 1612
955 262
956 198
956 17509
956 27844
956 75
956 14179
956 196
956 431
956 59418
956 44147
956 1239
956 3833
956 834
956 47032
956 27281
956 4842
956 20553
956 39819
956 3806
956 61989
956 155
956 5975
956 3513
956 40196
956 832
956 72180
956 6076
956 29269
956 3349
956 3232
956 1262
956 1101
956 370
956 2255
956 283
956 11353
956 1986
956 7164
956 2037
956 158
956 47354
956 12447
956 637
956 739
956 527
956 4314
956 6285
956 702
956 30714
956 29791
956 31558
956 2377
956 1380
956 4426
956 5336
956 36726
956 815
956 2443
956 52973
956 4773
956 15455
956 935
956 381
956 23333
956 41192
956 55
956 146
956 1226
956 29114
956 6180
956 3422
956 4680
956 1405
956 65096
956 34379
956 1244
956 2533
956 34737
956 4769
956 6758
956 9156
956 37946
956 425
956 236
956 154
957 983
957 897
957 4772
957 9495
957 382
957 115
957 231
957 157
957 37
957 4585
957 27
957 8842
957 386
957 68
957 240


982 24331
982 240
982 333
982 32714
982 20987
982 6595
982 2409
982 27956
982 106
983 1037
983 759
983 358
983 18703
983 25880
983 30203
983 3329
983 6054
983 1250
983 15152
983 702
983 1848
983 1412
983 8267
983 10433
983 10545
983 149
983 6261
983 15855
983 3633
983 24925
983 970
983 6697
983 24098
983 3326
983 2537
983 1370
983 554
983 9297
983 4614
983 460
983 1115
983 4
983 6976
983 628
983 2149
983 1414
983 897
983 4887
983 12221
983 1158
983 22230
983 13343
983 45
983 4528
983 34258
983 27693
983 14724
983 29528
983 20468
983 22475
983 22246
983 37360
983 26863
983 1998
983 1922
983 4173
983 371
983 26856
983 19727
983 459
983 43173
983 7508
983 10461
983 44309
983 2994
983 1107
983 662
984 6356
984 7713
984 32086
984 22851
984 21610
984 19885
984 18830
984 22989
984 1014
984 10069
984 1308
984 20151
984 10140
984 13
984 14584
984 366
984 4839
984 5371
984 33956
984 29661
984 10173
984 842
984 6340
984 26944
984 14728
984 27546
984 29750
985 38133
985 30491
985 36611
985 18199
9

In [87]:
assert isinstance(zip_dict, dict)
assert zip_dict[100] == 1502501


NameError: name 'zip_dict' is not defined

In [ ]:
# 3f) Explain this code excerpt 
# Note: you do not have to use this line of code at this point in the assignmnet.
#  It is one of the lines provided to you in 2e. Here, just write a quick comment on what it does. 

# In the cell below, explain in words what what the following line of code is doing:
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])

YOUR ANSWER HERE

In [ ]:
# 3g) Masking the Zip Codes 

# In this part, you should write a for loop, updating the df_users dataframe.
# Go through each user, and update their zip-code, to Safe Harbour specifications:
#   If the user is from a zip code for the which the
#     "Geographic Subdivision" is less than equal to 20000:
#        - Change the zip code to 0 
#   Otherwise:
#         - Change the zip code to be only the first 3 numbers of the full zip cide
# Do all this re-writting the zip_code columns of the 'df_users' DataFrame
#
# Hints:
#  - This will be several lines of code, looping through the DataFrame, 
#      getting each zip code, checking the geographic subdivision with 
#      the population in zip_dict, and settig the zip_code accordingly. 

# YOUR CODE HERE

#raise NotImplementedError()

In [ ]:
assert len(df_users) == 993
assert sum(df_users.zip == 0) == 7
assert df_users.loc[671, 'zip'] == 359


In [ ]:
# 3h) Save out the properly anonymized data to json file 

# Save out df_users as a json file, called 'real_anon_user_dat.json'

# YOUR CODE HERE

#raise NotImplementedError()

In [ ]:
assert isinstance(pd.read_json('real_anon_user_dat.json'), pd.DataFrame)

Congrats, you're done! The users identities are much more protected now. 

Submit this notebook file to TritonED.